In [72]:
import pandas as pd

filepath = "/Users/eshwarchandrasekharan/Desktop/repo/predict-pop/code/"
post_type = 'link'
links_meta = "link_metadata_2016_2017.csv"
links_1hr_stats = "link_1_hour_stats_2016_2017.csv"
links_48hr_stats = "link_48_hour_stats_2016_2017.csv"

# df = pd.read_csv("../code/" + post_type + '_metadata_06_2016.csv')
meta_df = pd.read_csv(filepath + links_meta)
stats_df = pd.read_csv(filepath + links_1hr_stats)
stats_df = stats_df.drop('created_at', 1)

train = pd.merge(meta_df, stats_df, on = ['external_id'], how = 'right')

###get 2 day stats
links_48hr_stats = "link_48_hour_stats_2016_2017.csv"
day2_stats_df = pd.read_csv(filepath + links_48hr_stats)
day2_stats_df = day2_stats_df[['external_id', 'stats_date', 'consumptions_by_type__link_clicks', 'stories_by_action_type__share']]
new_cols = ['external_id', 'twoday_stats_date', 'twoday_consumptions_by_type__link_clicks', 'twoday_stories_by_action_type__share']
day2_stats_df.columns = new_cols
train = pd.merge(train, day2_stats_df, on = ['external_id'], how = 'right')

###
temp = train
train_grouped = train.groupby(['buzz_id']).agg({'created_at':'min'})
train_grouped = train_grouped.reset_index()
train_grouped = train_grouped.rename(columns={'created_at':'first_post'})
df = pd.merge(train, train_grouped, how='left', on=['buzz_id'])
df = df[df['created_at'] == df['first_post']]
train = temp
###
df = df.drop('parent_external_id', 1)
df = df.dropna()
#get 48 hour stats

df = df.sort_values('stats_date', ascending = False).drop_duplicates(subset=['external_id'], keep = 'last')
df = df.sort_values('twoday_stats_date', ascending = False).drop_duplicates(subset=['external_id'], keep = 'last')
df.shape

(43638, 23)

In [74]:
df.to_csv('radshift_links_2016_2017.csv', index = None)
meta_df.shape, stats_df.shape, df.shape

((207497, 8), (161352, 13), (43638, 23))

In [86]:
train = pd.read_csv('radshift_links_2016_2017.csv')
train

In [44]:
meta_df['external_id'].shape, meta_df['external_id'].unique().shape

((207497,), (207497,))

In [79]:
df.columns

Index(['external_id', 'created_at', 'page', 'post_type', 'buzz_id', 'uri',
       'title', 'stats_ts', 'stats_date', 'consumptions_by_type__link_clicks',
       ...
       'Buzzed Feed', 'The Try Guys', 'BuzzFeed San Francisco',
       'BuzzFeed Sweaty', 'Proper Tasty', 'BuzzFeed BFF', 'BuzzFeed College',
       'BuzzReads', 'BuzzFeed Football', 'BuzzFeed Eugene'],
      dtype='object', length=107)

In [87]:
###get cross-posting info for videos!!!! 
import math 

filepath = "/Users/eshwarchandrasekharan/Desktop/repo/predict-pop/code/"
# train = pd.read_csv('../code/jan_may_2017_links_cross_posting_info.csv')
train = pd.read_csv('radshift_links_2016_2017.csv')

df = train.sort_values('created_at', ascending = False).drop_duplicates(subset=['buzz_id', 'external_id'], keep = 'last')

all_pages = list(df.page.unique())
print("Total pages considered: ", len(all_pages))

def get_cross_posted_pages(x):
        t = df[df.buzz_id == x].sort_values('created_at', ascending = 1)
        return t.page.unique()

df['cross_promotion_pages'] = df['buzz_id'].apply(get_cross_posted_pages)

def generate_cat_pages(x, y):
        if(y in x):
            return list(x).index(y) + 1
        else:
            return math.inf

###categorical features for pages where the link was shared
for pag in all_pages:
#     print("Getting for ", pag)
    df[pag] = df['cross_promotion_pages'].apply(generate_cat_pages, y = pag)

df.to_csv('radshift_links_cross_posting_info.csv', index = False)
df

In [88]:
###get the previous page the video was posted on!
train = df

def get_previous(X):
#     print("Start")
    buzzid = train[train['external_id'] == X]['buzz_id'].values[0]
    cross_posts = list(train[train['buzz_id'] == buzzid].sort_values(['created_at'], ascending = True)['external_id']) 
    print(X, ":", buzzid, ":", len(cross_posts))
#     print(cross_posts)
    index = cross_posts.index(X)
    if index == 0:
        return -1
#         return "-1"
    else:
        return cross_posts[index - 1]

df['previous_external_id'] = df['external_id'].apply(get_previous)

In [84]:
df[df['BuzzFeed'] == 2]

,external_id,created_at,page,post_type,buzz_id,uri,title,stats_ts,stats_date,consumptions_by_type__link_clicks,...,The Try Guys,BuzzFeed San Francisco,BuzzFeed Sweaty,Proper Tasty,BuzzFeed BFF,BuzzFeed College,BuzzReads,BuzzFeed Football,BuzzFeed Eugene,previous_external_id


In [ ]:
###get cross-posting info and previous-next page stats!!!! 
import math 

filepath = "/Users/eshwarchandrasekharan/Desktop/repo/predict-pop/code/"
# train = pd.read_csv('../code/jan_may_2017_links_cross_posting_info.csv')
train = pd.read_csv('radshift_links_2016_2017.csv')

df = train.sort_values('created_at', ascending = False).drop_duplicates(subset=['buzz_id', 'external_id'], keep = 'last')

all_pages = list(df.page.unique())
print("Total pages considered: ", len(all_pages))

def get_cross_posted_pages(x):
        t = df[df.buzz_id == x].sort_values('created_at', ascending = 1)
        return t.page.unique()

df['cross_promotion_pages'] = df['buzz_id'].apply(get_cross_posted_pages)

def generate_cat_pages(x, y):
        if(y in x):
            return list(x).index(y) + 1
        else:
            return math.inf

###categorical features for pages where the link was shared
for pag in all_pages:
#     print("Getting for ", pag)
    df[pag] = df['cross_promotion_pages'].apply(generate_cat_pages, y = pag)

df.to_csv('radshift_links_cross_posting_info.csv', index = False)
df
##################################
###get the previous page the video was posted on!
train = pd.read_csv('radshift_links_cross_posting_info.csv')

def get_previous(X):
#     print("Start")
    buzzid = train[train['external_id'] == X]['buzz_id'].values[0]
    cross_posts = list(train[train['buzz_id'] == buzzid].sort_values(['created_at'], ascending = True)['external_id']) 
    print(X, ":", buzzid, ":", len(cross_posts))
#     print(cross_posts)
    index = cross_posts.index(X)
    if index == 0:
        return -1
#         return "-1"
    else:
        return cross_posts[index - 1]

df['previous_external_id'] = df['external_id'].apply(get_previous)
df.to_csv('radshift_links_previous_page_info.csv', index = False)

##################################

all_df = pd.read_csv('radshift_links_cross_posting_info.csv')
all_pages = all_df['page'].unique()

alt_inf = len(all_pages) + 1
all_df = all_df.replace(math.inf, alt_inf)

pre_df = pd.read_csv('radshift_links_previous_page_info.csv')

pre_df = pre_df[~(pre_df['previous_external_id'] == "-1")]
pre_df = pre_df[['external_id','previous_external_id']]
pre_df.columns = ['next_external_id', 'external_id']
pre_df = pd.merge(pre_df, all_df, on="external_id", how = "inner")

features = [
       'next_external_id', 
       'external_id', 
       'page',
       'title',
        'buzz_id',
        'created_at',
       'stats_date',
       'consumptions_by_type__link_clicks',
       'consumptions_by_type__other_clicks',
       'reactions_like_total',
       'stories_by_action_type__comment',
       'stories_by_action_type__like',
       'stories_by_action_type__share', 
        'engaged_fan',
       'fan_reach', 
    'impressions',
       'impressions_fan',
        'twoday_stats_date',
       'twoday_consumptions_by_type__link_clicks',
       'twoday_stories_by_action_type__share',
        ]

for pg in all_pages:
    features.append(pg)

pre_df = pre_df[features]

post_df = all_df[['external_id', 'page', 'consumptions_by_type__link_clicks', 'stories_by_action_type__share']]
post_df.columns = ['next_external_id', 'next_page', 'next_clicks', 'next_shares']

train_df = pd.merge(pre_df, post_df, on="next_external_id", how = "inner")

train_df.to_csv('radshift-links-previous-to-next-page-info.csv', index = False)

In [89]:
###The code in the cell above is what you need!!! 

In [2]:
ts = "25 Cosas que SÍ SON REALES sobre los saqueos y las protestas ante el gasolinazo"

import nltk
from nltk.corpus import words

string = ts
# string = "21 Trucos para cuando estás tratando de comer más saludable"
# string = "It's Been 10 Years Since \"The Mysterious Ticking Noise\" And We're All Closer To Death"
tokens = nltk.word_tokenize(string)
eng_dict = words.words()

tot = 0
hits = 0

for word in tokens:
    tot += 1
    if word in eng_dict:
        hits += 1
print(hits, tot, hits/tot)

LookupError: 
**********************************************************************
  Resource 'corpora/words' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - '/Users/eshwarchandrasekharan/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************